In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import time
import glob
from tqdm import tqdm
import shutil
import random

In [ ]:


# # Set paths
# BASE_DIR = "/content/drive/MyDrive/Info_assurance/Dawn"
# DEST_DIR = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data"
# WEATHER_FOLDERS = ["Snow/images", "Sand/images", "Rain/images", "Fog/images"]
# PREDICTION_DIR = "/content/drive/MyDrive/Info_assurance/predictions/"

# # Create output directories
# for weather_folder in WEATHER_FOLDERS:
#     weather_name = weather_folder.split('/')[0]
#     os.makedirs(os.path.join(DEST_DIR, weather_name), exist_ok=True)
#     os.makedirs(os.path.join(DEST_DIR, weather_name, "images"), exist_ok=True)

# # Function to find all images in a directory
# def find_images_in_directory(directory, extensions=('jpg','jpeg','png')):
#     image_paths = []
#     for ext in extensions:
#         pattern = os.path.join(directory, f"*.{ext}")
#         image_paths.extend(glob.glob(pattern))
#         # Check uppercase as well
#         pattern_up = os.path.join(directory, f"*.{ext.upper()}")
#         image_paths.extend(glob.glob(pattern_up))
#     return sorted(image_paths)

# # Function to estimate atmospheric light and IcA for defog filter
# def estimate_atmospheric_light_and_ica(image):
#     """
#     Estimate atmospheric light (A) and IcA map using a dark-channel-based heuristic.
#     image: NumPy [H, W, 3], range [0,1].
#     Returns:
#       atmospheric_light: shape (3,)
#       IcA_map: shape (H, W)
#     """
#     dark_channel = np.min(image, axis=2)
#     flat_dark = dark_channel.ravel()
#     num_pixels = flat_dark.size
#     num_top_pixels = max(1, int(num_pixels * 0.001))  # top 0.1%

#     indices = np.argpartition(flat_dark, -num_top_pixels)[-num_top_pixels:]
#     y_indices, x_indices = np.unravel_index(indices, dark_channel.shape)

#     atmospheric_light = np.zeros(3, dtype=np.float32)
#     max_brightness = 0
#     for y, x in zip(y_indices, x_indices):
#         pix = image[y, x]
#         bright = np.mean(pix)
#         if bright > max_brightness:
#             max_brightness = bright
#             atmospheric_light = pix.astype(np.float32)

#     # Compute IcA = min_c(I_c / A_c)
#     norm_image = np.zeros_like(image)
#     for c in range(3):
#         if atmospheric_light[c] > 1e-6:
#             norm_image[..., c] = image[..., c] / atmospheric_light[c]
#         else:
#             norm_image[..., c] = image[..., c]

#     IcA_map = np.min(norm_image, axis=2)
#     return atmospheric_light, IcA_map

# # Helper functions for filters
# def rgb2lum(image):
#     r, g, b = image[..., 0], image[..., 1], image[..., 2]
#     return 0.27*r + 0.67*g + 0.06*b

# def lerp(a, b, l):
#     return a + l*(b - a)

# # Apply filters to images using predicted parameters
# def apply_filters_with_params(img, params):
#     """Apply all filters sequentially using predicted parameters"""
#     # Convert image to float32 and range [0, 1] if needed
#     if img.dtype != np.float32:
#         img = img.astype(np.float32)
#     if img.max() > 1.0:
#         img = img / 255.0

#     # Estimate atmospheric light and IcA for defog filter
#     A, IcA = estimate_atmospheric_light_and_ica(img)

#     # Extract parameters
#     fog_param = params['fog_param']
#     wb_params = np.array([params['wb_r'], params['wb_g'], params['wb_b']])
#     gamma_param = params['gamma']
#     tone_params = np.array([params[f'tone_{i}'] for i in range(8)])
#     contrast_param = params['contrast']
#     usm_param = params['usm']

#     # Apply defog filter
#     tx = 1.0 - fog_param * IcA
#     tx = np.clip(tx, 0.01, 1.0)
#     tx_3 = np.stack([tx, tx, tx], axis=2)
#     img = (img - A) / tx_3 + A

#     # Apply white balance filter
#     img = img * wb_params.reshape(1, 1, 3)

#     # Apply gamma filter
#     img = np.power(np.maximum(img, 1e-4), gamma_param)

#     # Apply tone filter (simplified)
#     tone_avg = np.mean(tone_params)
#     img = img * tone_avg

#     # Apply contrast filter
#     lum = rgb2lum(img)
#     lum = np.clip(lum, 0.0, 1.0)
#     contrast_lum = -np.cos(np.pi * lum) * 0.5 + 0.5
#     contrast_lum = np.expand_dims(contrast_lum, axis=2)
#     lum = np.expand_dims(lum, axis=2)
#     contrast_image = img / (lum + 1e-6) * contrast_lum
#     img = lerp(img, contrast_image, contrast_param)

#     # Apply USM filter
#     kernel = np.array([
#         [0, -1, 0],
#         [-1, 5, -1],
#         [0, -1, 0]
#     ], dtype=np.float32)

#     # Apply sharpening filter to each channel
#     sharpened = np.zeros_like(img)
#     for c in range(3):
#         sharpened[:,:,c] = cv2.filter2D(img[:,:,c], -1, kernel)

#     img = lerp(img, sharpened, usm_param)

#     # Clip to valid range
#     img = np.clip(img, 0.0, 1.0)

#     return img

# # Process each weather folder
# for weather_folder in WEATHER_FOLDERS:
#     weather_name = weather_folder.split('/')[0]
#     source_dir = os.path.join(BASE_DIR, weather_folder)
#     dest_dir = os.path.join(DEST_DIR, weather_name, "images")

#     # Check if source directory exists
#     if not os.path.exists(source_dir):
#         print(f"Warning: {source_dir} does not exist. Skipping...")
#         continue

#     # Load predictions from CSV
#     prediction_csv = os.path.join(PREDICTION_DIR, f"{weather_name}/images_predictions.csv")
#     if not os.path.exists(prediction_csv):
#         print(f"Warning: Prediction file {prediction_csv} not found. Skipping...")
#         continue

#     print(f"\nProcessing {weather_name} images...")

#     # Load predictions
#     predictions_df = pd.read_csv(prediction_csv)
#     print(f"Loaded {len(predictions_df)} predictions for {weather_name}")

#     # Create a dictionary of predictions for fast lookup
#     predictions_dict = {}
#     for _, row in predictions_df.iterrows():
#         predictions_dict[row['filename']] = row.to_dict()

#     # Find all images in the source directory
#     image_paths = find_images_in_directory(source_dir)

#     if not image_paths:
#         print(f"No images found in {source_dir}. Skipping...")
#         continue

#     print(f"Found {len(image_paths)} images in {weather_name} folder")

#     # Counter for matched files
#     matched_count = 0

#     # Process each image
#     for img_path in tqdm(image_paths, desc=f"Processing {weather_name}"):
#         img_filename = os.path.basename(img_path)

#         # Check if we have predictions for this image
#         if img_filename not in predictions_dict:
#             print(f"No prediction found for {img_filename}. Skipping...")
#             continue

#         matched_count += 1

#         # Read original image without resize for processing
#         orig_img = cv2.imread(img_path)
#         if orig_img is None:
#             print(f"Error reading image: {img_path}. Skipping...")
#             continue

#         orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)

#         # Get predictions for this image
#         params = predictions_dict[img_filename]

#         # Apply filters to the image
#         filtered_img = apply_filters_with_params(orig_img, params)

#         # Convert back to uint8 for saving
#         filtered_img_uint8 = (filtered_img * 255).astype(np.uint8)
#         filtered_img_bgr = cv2.cvtColor(filtered_img_uint8, cv2.COLOR_RGB2BGR)

#         # Save processed image
#         output_path = os.path.join(dest_dir, img_filename)
#         cv2.imwrite(output_path, filtered_img_bgr)

#     print(f"Processed {matched_count} images from {weather_name} folder")

# print("\nProcessing completed for all weather conditions!")


Processing Snow images...
Loaded 204 predictions for Snow
Found 204 images in Snow folder


Processing Snow: 100%|██████████| 204/204 [01:39<00:00,  2.04it/s]


Processed 204 images from Snow folder

Processing Sand images...
Loaded 323 predictions for Sand
Found 323 images in Sand folder


Processing Sand: 100%|██████████| 323/323 [02:16<00:00,  2.37it/s]


Processed 323 images from Sand folder

Processing Rain images...
Loaded 200 predictions for Rain
Found 200 images in Rain folder


Processing Rain: 100%|██████████| 200/200 [01:39<00:00,  2.01it/s]


Processed 200 images from Rain folder

Processing Fog images...
Loaded 300 predictions for Fog
Found 300 images in Fog folder


Processing Fog: 100%|██████████| 300/300 [02:16<00:00,  2.20it/s]

Processed 300 images from Fog folder

Processing completed for all weather conditions!


In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np
import cv2
import os
import glob
import pandas as pd
from tqdm import tqdm

In [ ]:


# Disable TF2 behaviors for a TF1.x style session/graph approach
tf.disable_v2_behavior()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging

# Set paths
BASE_DIR = "/content/drive/MyDrive/Info_assurance/Dawn"
DEST_DIR = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data"
WEATHER_FOLDERS = ["Snow/images", "Sand/images", "Rain/images", "Fog/images"]
PREDICTION_DIR = "/content/drive/MyDrive/Info_assurance/predictions/"

# Create output directories
for weather_folder in WEATHER_FOLDERS:
    weather_name = weather_folder.split('/')[0]
    os.makedirs(os.path.join(DEST_DIR, weather_name), exist_ok=True)
    os.makedirs(os.path.join(DEST_DIR, weather_name, "images"), exist_ok=True)

# Function to find all images in a directory
def find_images_in_directory(directory, extensions=('jpg','jpeg','png')):
    image_paths = []
    for ext in extensions:
        pattern = os.path.join(directory, f"*.{ext}")
        image_paths.extend(glob.glob(pattern))
        # Check uppercase as well
        pattern_up = os.path.join(directory, f"*.{ext.upper()}")
        image_paths.extend(glob.glob(pattern_up))
    return sorted(image_paths)

# Function to estimate atmospheric light and IcA for defog filter
def estimate_atmospheric_light_and_ica(image):
    """
    Estimate atmospheric light (A) and IcA map using a dark-channel-based heuristic.
    image: NumPy [H, W, 3], range [0,1].
    Returns:
      atmospheric_light: shape (3,)
      IcA_map: shape (H, W)
    """
    dark_channel = np.min(image, axis=2)
    flat_dark = dark_channel.ravel()
    num_pixels = flat_dark.size
    num_top_pixels = max(1, int(num_pixels * 0.001))  # top 0.1%

    indices = np.argpartition(flat_dark, -num_top_pixels)[-num_top_pixels:]
    y_indices, x_indices = np.unravel_index(indices, dark_channel.shape)

    atmospheric_light = np.zeros(3, dtype=np.float32)
    max_brightness = 0
    for y, x in zip(y_indices, x_indices):
        pix = image[y, x]
        bright = np.mean(pix)
        if bright > max_brightness:
            max_brightness = bright
            atmospheric_light = pix.astype(np.float32)

    # Compute IcA = min_c(I_c / A_c)
    norm_image = np.zeros_like(image)
    for c in range(3):
        if atmospheric_light[c] > 1e-6:
            norm_image[..., c] = image[..., c] / atmospheric_light[c]
        else:
            norm_image[..., c] = image[..., c]

    IcA_map = np.min(norm_image, axis=2)
    return atmospheric_light, IcA_map

##############################################################################
# DIP Filter Configuration
##############################################################################

class FilterConfig:
    def __init__(self):
        self.filters = [
            "DefogFilter", "WhiteBalanceFilter", "GammaFilter",
            "ToneFilter", "ContrastFilter", "SharpenFilter"
        ]
        self.defog_begin_param = 0
        self.wb_begin_param = 1
        self.gamma_begin_param = 4
        self.tone_begin_param = 5
        self.contrast_begin_param = 13
        self.usm_begin_param = 14
        self.curve_steps = 8

        self.gamma_range = 3.0
        self.wb_range = 1.1
        self.tone_curve_range = (0.5, 2.0)
        self.defog_range = (0.1, 1.0)
        self.usm_range = (0.0, 5.0)

        self.num_filter_parameters = 15

##############################################################################
# DIP Module Implementation
##############################################################################

def tanh_range(l, r, initial=None):
    """Map input to [l, r] using tanh."""
    def activation(x):
        if initial is None:
            return (tf.tanh(x) * 0.5 + 0.5)*(r - l) + l
        else:
            target = (initial - l)/(r - l)*2.0 - 1.0
            target = tf.clip_by_value(target, -0.999, 0.999)
            return (tf.tanh(x + tf.atanh(target)) * 0.5 + 0.5)*(r - l) + l
    return activation

def rgb2lum(image):
    r, g, b = image[..., 0], image[..., 1], image[..., 2]
    return 0.27*r + 0.67*g + 0.06*b

def lerp(a, b, l):
    return a + l*(b - a)

class Filter:
    def __init__(self, cfg):
        self.cfg = cfg
        self.short_name = None
        self.begin_filter_parameter = None
        self.num_filter_parameters = None

    def filter_param_regressor(self, features):
        raise NotImplementedError

    def process(self, img, param, defog_A=None, IcA=None):
        raise NotImplementedError

class DefogFilter(Filter):
    """Defog: J = (I - A)/t + A, t = 1 - param*IcA."""
    def __init__(self, cfg):
        super().__init__(cfg)
        self.short_name = 'DF'
        self.begin_filter_parameter = cfg.defog_begin_param
        self.num_filter_parameters = 1

    def filter_param_regressor(self, features):
        return tanh_range(*self.cfg.defog_range)(features)

    def process(self, img, param, defog_A, IcA):
        # param: [batch,1], IcA: [batch,H,W], defog_A: [batch,3]
        rank = tf.rank(IcA)
        shape = tf.shape(IcA)

        # If IcA is [batch,H,W,1], squeeze
        IcA = tf.cond(
            tf.logical_and(tf.equal(rank, 4), tf.equal(shape[-1], 1)),
            lambda: tf.squeeze(IcA, axis=-1),
            lambda: IcA
        )
        param_reshaped = tf.reshape(param, [-1,1,1])
        tx = 1.0 - param_reshaped * IcA
        tx = tf.clip_by_value(tx, 0.01, 1.0)
        tx_expanded = tf.expand_dims(tx, axis=-1)  # [batch,H,W,1]
        tx_3 = tf.tile(tx_expanded, [1,1,1,3])

        A_expanded = tf.reshape(defog_A, [-1,1,1,3])
        return (img - A_expanded)/tx_3 + A_expanded

class ImprovedWhiteBalanceFilter(Filter):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.short_name = 'W'
        self.begin_filter_parameter = cfg.wb_begin_param
        self.num_filter_parameters = 3

    def filter_param_regressor(self, features):
        mask = tf.constant([[0,1,1]], dtype=tf.float32)
        features = features*mask
        color_scaling = tf.exp(tanh_range(-0.5,0.5)(features))
        r, g, b = color_scaling[:,0], color_scaling[:,1], color_scaling[:,2]
        lum = 0.27*r + 0.67*g + 0.06*b
        lum = tf.expand_dims(lum, axis=1)
        color_scaling_norm = color_scaling/(lum + 1e-5)
        return color_scaling_norm

    def process(self, img, param, defog_A=None, IcA=None):
        param_expanded = tf.reshape(param, [-1,1,1,3])
        return img * param_expanded

class GammaFilter(Filter):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.short_name = 'G'
        self.begin_filter_parameter = cfg.gamma_begin_param
        self.num_filter_parameters = 1

    def filter_param_regressor(self, features):
        log_gamma_range = np.log(self.cfg.gamma_range)
        return tf.exp(tanh_range(-log_gamma_range, log_gamma_range)(features))

    def process(self, img, param, defog_A=None, IcA=None):
        param_tiled = tf.tile(param, [1,3])
        param_expanded = tf.reshape(param_tiled, [-1,1,1,3])
        img_safe = tf.maximum(img, 1e-4)
        return tf.pow(img_safe, param_expanded)

class ToneFilter(Filter):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.short_name = 'T'
        self.begin_filter_parameter = cfg.tone_begin_param
        self.num_filter_parameters = cfg.curve_steps

    def filter_param_regressor(self, features):
        return tanh_range(*self.cfg.tone_curve_range)(features)

    def process(self, img, param, defog_A=None, IcA=None):
        tone_param = tf.reduce_mean(param, axis=1, keepdims=True)
        tone_param_expanded = tf.reshape(tone_param, [-1,1,1,1])
        return img * tone_param_expanded

class ContrastFilter(Filter):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.short_name = 'Ct'
        self.begin_filter_parameter = cfg.contrast_begin_param
        self.num_filter_parameters = 1

    def filter_param_regressor(self, features):
        return tf.tanh(features)

    def process(self, img, param, defog_A=None, IcA=None):
        lum = rgb2lum(img)
        lum = tf.clip_by_value(lum, 0.0, 1.0)
        contrast_lum = -tf.cos(np.pi*lum)*0.5 + 0.5
        lum_expanded = tf.expand_dims(lum, axis=-1)
        contrast_lum_expanded = tf.expand_dims(contrast_lum, axis=-1)
        contrast_image = img/(lum_expanded+1e-6)*contrast_lum_expanded

        blend = tf.reshape(param, [-1,1,1,1])
        return lerp(img, contrast_image, blend)

class UsmFilter(Filter):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.short_name = 'UF'
        self.begin_filter_parameter = cfg.usm_begin_param
        self.num_filter_parameters = 1

    def filter_param_regressor(self, features):
        return tanh_range(*self.cfg.usm_range)(features)

    def process(self, img, param, defog_A=None, IcA=None):
        kernel = tf.constant([
            [0, -1, 0],
            [-1, 5, -1],
            [0, -1, 0]
        ], dtype=tf.float32)
        kernel = kernel[:,:,None,None]

        padded = tf.pad(img, [[0,0],[1,1],[1,1],[0,0]], mode='REFLECT')
        sharpened_channels = []
        for c in range(3):
            channel = padded[..., c:c+1]
            sharp_c = tf.nn.conv2d(channel, kernel, strides=[1,1,1,1], padding='VALID')
            sharpened_channels.append(sharp_c)
        sharpened = tf.concat(sharpened_channels, axis=3)

        amount = tf.reshape(param, [-1,1,1,1])
        return lerp(img, sharpened, amount)

class DIPModule:
    def __init__(self, session=None):
        self.cfg = FilterConfig()

        if session is None:
            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.session = tf.Session(config=config)
            self.should_close_session = True
        else:
            self.session = session
            self.should_close_session = False

        # Build ops in default graph
        self._build_model()
        init_op = tf.global_variables_initializer()
        self.session.run(init_op)

    def _build_model(self):
        self.input_highres = tf.placeholder(
            tf.float32, [None,None,None,3], name='input_highres')
        self.input_params = tf.placeholder(
            tf.float32, [None, self.cfg.num_filter_parameters], name='input_params')
        self.input_defog_A = tf.placeholder(
            tf.float32, [None,3], name='input_defog_A')
        self.input_IcA = tf.placeholder(
            tf.float32, [None,None,None], name='input_IcA')

        self.filters = [
            DefogFilter(self.cfg),
            ImprovedWhiteBalanceFilter(self.cfg),
            GammaFilter(self.cfg),
            ToneFilter(self.cfg),
            ContrastFilter(self.cfg),
            UsmFilter(self.cfg)
        ]

        current_image = self.input_highres
        self.filter_outputs = []

        for f in self.filters:
            sidx = f.begin_filter_parameter
            eidx = sidx + f.num_filter_parameters
            f_params = self.input_params[:, sidx:eidx]

            if f.short_name == 'DF':
                current_image = f.process(
                    current_image, f_params, self.input_defog_A, self.input_IcA
                )
            else:
                current_image = f.process(current_image, f_params)

            self.filter_outputs.append(current_image)

        self.output_image = current_image

    def process_image(self, highres_batch, params_batch,
                      defog_A_batch=None, IcA_batch=None):
        bsz, H, W, _ = highres_batch.shape

        if defog_A_batch is None:
            defog_A_batch = np.ones((bsz,3), dtype=np.float32)
        if IcA_batch is None:
            IcA_batch = np.zeros((bsz,H,W), dtype=np.float32)
        # If user gave [bsz,H,W,1], squeeze it in numpy
        if IcA_batch.ndim == 4 and IcA_batch.shape[-1] == 1:
            IcA_batch = IcA_batch[...,0]

        return self.session.run(
            self.output_image,
            feed_dict={
                self.input_highres: highres_batch,
                self.input_params: params_batch,
                self.input_defog_A: defog_A_batch,
                self.input_IcA: IcA_batch
            }
        )

    def close(self):
        if self.should_close_session and self.session is not None:
            self.session.close()
            self.session = None

# Function to convert CSV parameters to the format expected by DIPModule
def convert_predictions_to_params(pred_row):
    """Convert CSV prediction row to parameter array for DIPModule"""
    params = np.zeros(15, dtype=np.float32)

    # fog_param (index 0)
    params[0] = pred_row['fog_param']

    # wb_r, wb_g, wb_b (indices 1, 2, 3)
    params[1] = pred_row['wb_r']
    params[2] = pred_row['wb_g']
    params[3] = pred_row['wb_b']

    # gamma (index 4)
    params[4] = pred_row['gamma']

    # tone_0 to tone_7 (indices 5 to 12)
    for i in range(8):
        params[5 + i] = pred_row[f'tone_{i}']

    # contrast (index 13)
    params[13] = pred_row['contrast']

    # usm (index 14)
    params[14] = pred_row['usm']

    return params

# Main function to apply filters to images using predicted parameters
def process_images_with_dip():
    # Initialize the DIP module
    dip_module = DIPModule()

    try:
        # Process each weather folder
        for weather_folder in WEATHER_FOLDERS:
            weather_name = weather_folder.split('/')[0]
            source_dir = os.path.join(BASE_DIR, weather_folder)
            dest_dir = os.path.join(DEST_DIR, weather_name, "images")

            # Check if source directory exists
            if not os.path.exists(source_dir):
                print(f"Warning: {source_dir} does not exist. Skipping...")
                continue

            # Load predictions from CSV
            prediction_csv = os.path.join(PREDICTION_DIR, f"{weather_name}/images_predictions.csv")
            if not os.path.exists(prediction_csv):
                print(f"Warning: Prediction file {prediction_csv} not found. Skipping...")
                continue

            print(f"\nProcessing {weather_name} images...")

            # Load predictions
            predictions_df = pd.read_csv(prediction_csv)
            print(f"Loaded {len(predictions_df)} predictions for {weather_name}")

            # Create a dictionary of predictions for fast lookup
            predictions_dict = {}
            for _, row in predictions_df.iterrows():
                predictions_dict[row['filename']] = row.to_dict()

            # Find all images in the source directory
            image_paths = find_images_in_directory(source_dir)

            if not image_paths:
                print(f"No images found in {source_dir}. Skipping...")
                continue

            print(f"Found {len(image_paths)} images in {weather_name} folder")

            # Counter for matched files
            matched_count = 0

            # Process each image
            for img_path in tqdm(image_paths, desc=f"Processing {weather_name}"):
                img_filename = os.path.basename(img_path)

                # Check if we have predictions for this image
                if img_filename not in predictions_dict:
                    print(f"No prediction found for {img_filename}. Skipping...")
                    continue

                matched_count += 1

                # Read original image
                orig_img = cv2.imread(img_path)
                if orig_img is None:
                    print(f"Error reading image: {img_path}. Skipping...")
                    continue

                # Convert BGR to RGB and normalize
                orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB) / 255.0

                # Get predictions for this image
                pred_params = predictions_dict[img_filename]

                # Convert predictions to DIP module format
                dip_params = convert_predictions_to_params(pred_params)

                # Estimate atmospheric light and IcA for defog filter
                defog_A, IcA_map = estimate_atmospheric_light_and_ica(orig_img)

                # Prepare inputs as batches (add batch dimension)
                highres_batch = np.expand_dims(orig_img, axis=0)
                params_batch = np.expand_dims(dip_params, axis=0)
                defog_A_batch = np.expand_dims(defog_A, axis=0)
                IcA_batch = np.expand_dims(IcA_map, axis=0)

                # Process image through DIP module
                processed_batch = dip_module.process_image(
                    highres_batch, params_batch, defog_A_batch, IcA_batch
                )

                # Get processed image (remove batch dimension)
                processed_img = processed_batch[0]

                # Convert back to uint8 BGR for saving
                processed_img_uint8 = (np.clip(processed_img, 0, 1) * 255).astype(np.uint8)
                processed_img_bgr = cv2.cvtColor(processed_img_uint8, cv2.COLOR_RGB2BGR)

                # Save processed image
                output_path = os.path.join(dest_dir, img_filename)
                cv2.imwrite(output_path, processed_img_bgr)

            print(f"Processed {matched_count} images from {weather_name} folder")

    finally:
        # Clean up resources
        dip_module.close()
        print("DIP module resources cleaned up")



In [ ]:

process_images_with_dip()


Processing Snow images...
Loaded 204 predictions for Snow
Found 204 images in Snow folder


Processing Snow: 100%|██████████| 204/204 [02:12<00:00,  1.54it/s]


Processed 204 images from Snow folder

Processing Sand images...
Loaded 323 predictions for Sand
Found 323 images in Sand folder


Processing Sand: 100%|██████████| 323/323 [02:57<00:00,  1.82it/s]


Processed 323 images from Sand folder

Processing Rain images...
Loaded 200 predictions for Rain
Found 200 images in Rain folder


Processing Rain: 100%|██████████| 200/200 [02:06<00:00,  1.57it/s]


Processed 200 images from Rain folder

Processing Fog images...
Loaded 300 predictions for Fog
Found 300 images in Fog folder


Processing Fog: 100%|██████████| 300/300 [03:04<00:00,  1.63it/s]

Processed 300 images from Fog folder
DIP module resources cleaned up


In [ ]:

# 1) Define your old→new ID mapping here
id_mapping = {
    1: 0,  # person
    2: 1,  # car
    3: 2,  # car
    4: 3,  # motorcycle
    5: 4,  # car
    6: 5,  # bus
    7: 6,  # car
    8: 7   # truck
}

def remap_labels_in_folder(labels_dir, overwrite=True):
    """
    Remap old class IDs to new 0-based YOLO IDs for all .txt files in a given folder.

    Args:
        labels_dir (str): Path to the folder containing .txt label files.
        overwrite (bool): If True, modifies files in-place. Otherwise,
                          creates new files with suffix '_remapped'.
    """
    for filename in os.listdir(labels_dir):
        if not filename.endswith(".txt"):
            continue

        txt_path = os.path.join(labels_dir, filename)

        with open(txt_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                # Skip empty or malformed lines
                continue

            # Parse old class ID
            old_id = int(parts[0])

            # If this old_id is in our mapping, replace it
            if old_id in id_mapping:
                new_id = id_mapping[old_id]
                parts[0] = str(new_id)
                new_line = " ".join(parts) + "\n"
                new_lines.append(new_line)
            else:
                # If an ID is not in the mapping, decide how to handle:
                # - skip it
                # - or keep the old ID
                # - or raise an error
                # Here we'll just skip that line:
                print(f"Skipping unknown ID {old_id} in file {filename}")

        # Write out the updated lines
        if overwrite:
            # Overwrite the original file
            with open(txt_path, "w") as f_out:
                f_out.writelines(new_lines)
        else:
            # Create a new file with '_remapped' suffix
            base, ext = os.path.splitext(txt_path)
            new_txt_path = base + "_remapped" + ext
            with open(new_txt_path, "w") as f_out:
                f_out.writelines(new_lines)



In [ ]:


# 2) Path to your DAWN data directory
data_dir = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data"

# 3) Folders to process (each has images/ and labels/)
categories = ["Fog", "Rain", "Sand", "Snow"]

# 4) Run the remapping on each labels/ folder
for cat in categories:
    labels_path = os.path.join(data_dir, cat, "labels")
    if os.path.isdir(labels_path):
        print(f"Remapping labels in: {labels_path}")
        remap_labels_in_folder(labels_path, overwrite=True)
    else:
        print(f"Skipping: {labels_path} does not exist")

Remapping labels in: /content/drive/MyDrive/Info_assurance/synthetic_dawn_data/Fog/labels
Remapping labels in: /content/drive/MyDrive/Info_assurance/synthetic_dawn_data/Rain/labels
Remapping labels in: /content/drive/MyDrive/Info_assurance/synthetic_dawn_data/Sand/labels
Remapping labels in: /content/drive/MyDrive/Info_assurance/synthetic_dawn_data/Snow/labels


In [ ]:


def split_data(
    source_dir,
    dest_dir,
    categories=["Fog", "Rain", "Sand", "Snow"],
    train_ratio=0.7,
    val_ratio=0.1,
    seed=42
):
    """
    Splits images and corresponding YOLO label files into train, val, and test sets.

    Args:
        source_dir (str): Path to the original data directory containing subfolders (Fog, Rain, etc.).
        dest_dir (str): Path to the directory where the split data will be stored.
        categories (list): List of category folder names to process.
        train_ratio (float): Fraction of data to use for training.
        val_ratio (float): Fraction of data to use for validation.
        seed (int): Random seed for reproducibility.
    """
    random.seed(seed)

    # The remainder after train_ratio and val_ratio is the test ratio
    # e.g. if train_ratio=0.7 and val_ratio=0.1 => test_ratio=0.2
    test_ratio = 1.0 - (train_ratio + val_ratio)
    if test_ratio < 0:
        raise ValueError("Sum of train_ratio and val_ratio cannot exceed 1.0")

    for category in categories:
        print(f"Processing category: {category}")

        # Paths for images and labels in the source directory
        category_path = os.path.join(source_dir, category)
        images_path = os.path.join(category_path, "images")
        labels_path = os.path.join(category_path, "labels")

        # Collect all image filenames in the category
        all_images = [f for f in os.listdir(images_path) if f.endswith(".jpg")]
        random.shuffle(all_images)

        # Calculate split indices
        total_images = len(all_images)
        train_end = int(total_images * train_ratio)
        val_end = train_end + int(total_images * val_ratio)

        # Split the list of images into train/val/test
        train_images = all_images[:train_end]
        val_images   = all_images[train_end:val_end]
        test_images  = all_images[val_end:]

        # Prepare output directories for train/val/test images & labels
        train_images_out = os.path.join(dest_dir, category, "train", "images")
        train_labels_out = os.path.join(dest_dir, category, "train", "labels")

        val_images_out   = os.path.join(dest_dir, category, "val", "images")
        val_labels_out   = os.path.join(dest_dir, category, "val", "labels")

        test_images_out  = os.path.join(dest_dir, category, "test", "images")
        test_labels_out  = os.path.join(dest_dir, category, "test", "labels")

        os.makedirs(train_images_out, exist_ok=True)
        os.makedirs(train_labels_out, exist_ok=True)
        os.makedirs(val_images_out, exist_ok=True)
        os.makedirs(val_labels_out, exist_ok=True)
        os.makedirs(test_images_out, exist_ok=True)
        os.makedirs(test_labels_out, exist_ok=True)

        # Helper function to copy image/label pairs
        def copy_data(image_list, images_out, labels_out):
            for img_file in image_list:
                # Copy image
                src_img = os.path.join(images_path, img_file)
                dst_img = os.path.join(images_out, img_file)
                shutil.copy2(src_img, dst_img)

                # Copy corresponding label (same base name, .txt extension)
                label_file = os.path.splitext(img_file)[0] + ".txt"
                src_label = os.path.join(labels_path, label_file)
                if os.path.exists(src_label):
                    dst_label = os.path.join(labels_out, label_file)
                    shutil.copy2(src_label, dst_label)

        # Copy train/val/test data
        copy_data(train_images, train_images_out, train_labels_out)
        copy_data(val_images,   val_images_out,   val_labels_out)
        copy_data(test_images,  test_images_out,  test_labels_out)

        print(f"Finished processing {category}:")
        print(f"  Train images: {len(train_images)}")
        print(f"  Val images:   {len(val_images)}")
        print(f"  Test images:  {len(test_images)}\n")



In [ ]:

original_data_dir = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data"
split_data_dir = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/split_dawn_data"

split_data(
    source_dir=original_data_dir,
    dest_dir=split_data_dir,
    categories=["Fog", "Rain", "Sand", "Snow"],
    train_ratio=0.7,  # 70% training
    val_ratio=0.1,    # 10% validation
    seed=42
)

Processing category: Fog
Finished processing Fog:
  Train images: 210
  Val images:   30
  Test images:  60

Processing category: Rain
Finished processing Rain:
  Train images: 140
  Val images:   20
  Test images:  40

Processing category: Sand
Finished processing Sand:
  Train images: 226
  Val images:   32
  Test images:  65

Processing category: Snow
Finished processing Snow:
  Train images: 142
  Val images:   20
  Test images:  42



In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import os
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model =  YOLO("yolo11s.pt")

100%|██████████| 18.4M/18.4M [00:00<00:00, 104MB/s] 


In [ ]:
# Display model information (optional)
model.info()

YOLO11s summary: 181 layers, 9,458,752 parameters, 0 gradients, 21.7 GFLOPs


(181, 9458752, 0, 21.718374400000002)

#Fog

In [ ]:

# Train the model on the yolo8 example dataset for 10 epochs
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
results = model.train(data= "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/split_dawn_data/fog.yaml", epochs=10) # , imgsz=640


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/Info_assurance/fog.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

100%|██████████| 755k/755k [00:00<00:00, 27.2MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 98.2MB/s]


AMP: checks passed ✅


...

      10/10      4.16G      1.241      1.107      1.136          5        640: 100%|██████████| 14/14 [00:04<00:00,  3.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.53it/s]

                   all         30        165      0.432      0.646      0.562      0.344



10 epochs completed in 0.023 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.66it/s]


                   all         30        165      0.431      0.642      0.562      0.346
                person          5          7      0.504          1       0.91      0.571
                   car         28        130      0.673      0.919      0.909      0.638
            motorcycle          4          5      0.478      0.374      0.208     0.0513
                   bus          5          6      0.146        0.5      0.391      0.288
                 truck          4         17      0.353      0.417      0.391       0.18
Speed: 0.2ms preprocess, 4.6ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train


In [ ]:


source = "/content/runs"
destination = "/content/drive/MyDrive/Info_assurance/models/baseline_10/fog"

# If the destination folder already exists and you want to merge/overwrite,
# pass dirs_exist_ok=True (available in Python 3.8+)
shutil.copytree(src=source, dst=destination, dirs_exist_ok=True)


'/content/drive/MyDrive/Info_assurance/models/baseline_10/fog'

## Remove runs

In [ ]:
directory = "/content/runs"

if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Directory '{directory}' has been removed.")
else:
    print(f"Directory '{directory}' does not exist.")

In [ ]:
# Load the trained/best weights
model_path = "/content/drive/MyDrive/Info_assurance/models/baseline_10/fog/detect/train/weights/best.pt"
model = YOLO(model_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Info_assurance/models/baseline_10/fog/runs/detect/train/weights/best.pt'

In [ ]:
results = model.predict(source="/content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/test/images",
                        conf=0.25,
                        save=True)


In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt

In [ ]:

results = model("/content/drive/MyDrive/Walid/Run1/v0/test/images/morning1_mp4-0393_jpg.rf.97c6d913fe73be772de96b8341c42ba5.jpg")



image 1/1 /content/drive/MyDrive/Walid/Run1/v0/test/images/morning1_mp4-0393_jpg.rf.97c6d913fe73be772de96b8341c42ba5.jpg: 384x640 6 cars, 2 trucks, 208.0ms
Speed: 5.1ms preprocess, 208.0ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)


## Fog


In [ ]:

# Train the model on the yolo8 example dataset for 10 epochs
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
results = model.train(data= "/content/drive/MyDrive/Info_assurance/fog.yaml", epochs=100) # , imgsz=640


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/Info_assurance/fog.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

train: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/train/labels.cache... 210 images, 1 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/val/labels.cache... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]


Plotting labels to runs/detect/train34/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train34
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      4.12G     0.7756     0.4255     0.8927         33        640: 100%|██████████| 14/14 [00:04<00:00,  3.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.37it/s]

                   all         30        165      0.766      0.696      0.763       0.48



100 epochs completed in 0.185 hours.
Optimizer stripped from runs/detect/train34/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train34/weights/best.pt, 19.2MB

Validating runs/detect/train34/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.77it/s]


                   all         30        165      0.754      0.711      0.782      0.502
                person          5          7      0.626      0.857      0.924      0.548
                   car         28        130      0.868        0.9      0.925      0.681
            motorcycle          4          5      0.799      0.794       0.84      0.579
                   bus          5          6      0.738      0.333      0.543      0.407
                 truck          4         17       0.74      0.671       0.68      0.296
Speed: 0.2ms preprocess, 5.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train34


In [ ]:


source = "/content/runs"
destination = "/content/drive/MyDrive/Info_assurance/models/baseline_100/fog"

# If the destination folder already exists and you want to merge/overwrite,
# pass dirs_exist_ok=True (available in Python 3.8+)
shutil.copytree(src=source, dst=destination, dirs_exist_ok=True)


'/content/drive/MyDrive/Info_assurance/models/baseline_100/fog'

## Remove runs

In [ ]:
directory = "/content/runs"

if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Directory '{directory}' has been removed.")
else:
    print(f"Directory '{directory}' does not exist.")

Directory '/content/runs' has been removed.


In [ ]:
# Load the trained/best weights
model_path = "/content/drive/MyDrive/Info_assurance/models/baseline_100/fog/detect/train34/weights/best.pt"
model = YOLO(model_path)

In [ ]:
# results = model.predict(source="/content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/test/images",
#                         conf=0.25,
#                         save=True)


In [ ]:
# for r in results:
#     boxes = r.boxes  # list of boxes
#     for box in boxes:
#         cls_id = int(box.cls[0])
#         conf = float(box.conf[0])
#         # x1, y1, x2, y2 = box.xyxy[0]
#         print(f"Detected class {cls_id} with confidence {conf}")


In [ ]:
# Evaluate on test set
metrics = model.val(data="/content/drive/MyDrive/Info_assurance/fog.yaml",
                    split="test")  # explicitly evaluate the test split


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/test/labels.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.59it/s]


                   all         60        379      0.707      0.722      0.755      0.458
                person          8         18      0.834      0.778       0.81      0.481
               bicycle          1          2      0.533          1      0.828      0.465
                   car         59        306      0.876      0.856        0.9      0.629
            motorcycle          5          7      0.851      0.714      0.897      0.465
                   bus          7         11      0.529      0.412      0.376      0.305
                 truck         22         35      0.619      0.571      0.716        0.4
Speed: 4.7ms preprocess, 7.9ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val6


In [ ]:
print(metrics)  # or metrics.keys()


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cea464df2d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

# Rain

In [ ]:

# Train the model on the yolo8 example dataset for 10 epochs
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
results = model.train(data= "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/rain.yaml", epochs=100) # , imgsz=640


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/Info_assurance/rain.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Rain/train/labels.cache... 140 images, 0 backgrounds, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Rain/val/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to runs/detect/train34/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train34
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      4.24G       0.71     0.3846     0.8617        111        640: 100%|██████████| 9/9 [00:02<00:00,  3.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.03it/s]

                   all         20        163      0.909      0.401      0.567      0.285



100 epochs completed in 0.118 hours.
Optimizer stripped from runs/detect/train34/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train34/weights/best.pt, 19.2MB

Validating runs/detect/train34/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.89it/s]


                   all         20        163      0.921      0.361      0.651      0.352
                person          4          7      0.977      0.429      0.765      0.418
                   car         19        133      0.888      0.779       0.88      0.617
            motorcycle          2          3          1          0      0.777      0.222
                   bus          5          5          1          0      0.116      0.105
                 truck          8         15      0.741        0.6      0.714      0.397
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train34


In [ ]:

source = "/content/runs"
destination = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/models/baseline_100/rain"

# If the destination folder already exists and you want to merge/overwrite,
# pass dirs_exist_ok=True (available in Python 3.8+)
shutil.copytree(src=source, dst=destination, dirs_exist_ok=True)

'/content/drive/MyDrive/Info_assurance/models/baseline_100/rain'

## Remove runs

In [ ]:
directory = "/content/runs"

if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Directory '{directory}' has been removed.")
else:
    print(f"Directory '{directory}' does not exist.")

Directory '/content/runs' has been removed.


In [ ]:
# Load the trained/best weights
model_path = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/models/baseline_100/rain/detect/train34/weights/best.pt"
model = YOLO(model_path)

In [ ]:
# results = model.predict(source="/content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/test/images",
#                         conf=0.25,
#                         save=True)


In [ ]:
# for r in results:
#     boxes = r.boxes  # list of boxes
#     for box in boxes:
#         cls_id = int(box.cls[0])
#         conf = float(box.conf[0])
#         # x1, y1, x2, y2 = box.xyxy[0]
#         print(f"Detected class {cls_id} with confidence {conf}")


In [ ]:
# Evaluate on test set
metrics = model.val(data="/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/rain.yaml",
                    split="test")  # explicitly evaluate the test split


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Rain/test/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.59it/s]


                   all         40        338      0.761      0.566      0.551      0.323
                person          4          4      0.476       0.75      0.552      0.171
                   car         40        285      0.837      0.923      0.934      0.657
                   bus          3          3          1          0          0          0
                 truck         14         46      0.731      0.589      0.717      0.462
Speed: 5.5ms preprocess, 8.5ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val7


In [ ]:
print(metrics)  # or metrics.keys()


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 2, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ce9f8dd8e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

##Sand

In [ ]:

# Train the model on the yolo8 example dataset for 10 epochs
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
results = model.train(data= "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/sand.yaml", epochs=100) # , imgsz=640


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/Info_assurance/sand.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Sand/train/labels... 226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 226/226 [01:27<00:00,  2.58it/s]

train: New cache created: /content/drive/MyDrive/Info_assurance/split_dawn_data/Sand/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Sand/val/labels... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:28<00:00,  1.13it/s]

val: New cache created: /content/drive/MyDrive/Info_assurance/split_dawn_data/Sand/val/labels.cache


...

    100/100      4.15G     0.7039      0.425     0.8881         15        640: 100%|██████████| 15/15 [00:05<00:00,  2.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

                   all         32        261      0.837      0.699      0.822      0.577



100 epochs completed in 0.194 hours.
Optimizer stripped from runs/detect/train34/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train34/weights/best.pt, 19.2MB

Validating runs/detect/train34/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


                   all         32        261      0.875      0.727      0.831      0.593
                person          9         19      0.884      0.802      0.867       0.53
               bicycle          2          2          1        0.8      0.995      0.798
                   car         31        211      0.911      0.774      0.867      0.597
            motorcycle          6         11      0.924      0.727      0.925      0.552
                   bus          3          3      0.783      0.667      0.747      0.649
                 truck         10         15      0.748      0.594      0.587      0.432
Speed: 0.2ms preprocess, 3.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train34


In [ ]:

source = "/content/runs"
destination = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/models/baseline_100/sand"

# If the destination folder already exists and you want to merge/overwrite,
# pass dirs_exist_ok=True (available in Python 3.8+)
shutil.copytree(src=source, dst=destination, dirs_exist_ok=True)

'/content/drive/MyDrive/Info_assurance/models/baseline_100/sand'

## Remove runs

In [ ]:
directory = "/content/runs"

if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Directory '{directory}' has been removed.")
else:
    print(f"Directory '{directory}' does not exist.")

Directory '/content/runs' has been removed.


In [ ]:
# Load the trained/best weights
model_path = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/models/baseline_100/sand/detect/train34/weights/best.pt"
model = YOLO(model_path)

In [ ]:
# results = model.predict(source="/content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/test/images",
#                         conf=0.25,
#                         save=True)


In [ ]:
# for r in results:
#     boxes = r.boxes  # list of boxes
#     for box in boxes:
#         cls_id = int(box.cls[0])
#         conf = float(box.conf[0])
#         # x1, y1, x2, y2 = box.xyxy[0]
#         print(f"Detected class {cls_id} with confidence {conf}")


In [ ]:
# Evaluate on test set
metrics = model.val(data="/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/sand.yaml",
                    split="test")  # explicitly evaluate the test split


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Sand/test/labels... 65 images, 0 backgrounds, 0 corrupt: 100%|██████████| 65/65 [00:59<00:00,  1.09it/s]

val: New cache created: /content/drive/MyDrive/Info_assurance/split_dawn_data/Sand/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.97it/s]


                   all         65        430      0.733      0.587      0.667      0.465
                person          9         11      0.563      0.545      0.556      0.433
                   car         63        363      0.916      0.818      0.919      0.644
                   bus         10         12      0.625      0.417      0.472       0.31
                 truck         23         44      0.828      0.568      0.722      0.472
Speed: 5.0ms preprocess, 9.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val8


In [ ]:
print(metrics)  # or metrics.keys()


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 2, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ceae3cc8e90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

##Snow

In [ ]:

# Train the model on the yolo8 example dataset for 10 epochs
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
results = model.train(data= "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/snow.yaml", epochs=100) # , imgsz=640


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/Info_assurance/snow.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Snow/train/labels... 142 images, 0 backgrounds, 0 corrupt: 100%|██████████| 142/142 [00:55<00:00,  2.54it/s]

train: New cache created: /content/drive/MyDrive/Info_assurance/split_dawn_data/Snow/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Snow/val/labels... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:15<00:00,  1.33it/s]

val: New cache created: /content/drive/MyDrive/Info_assurance/split_dawn_data/Snow/val/labels.cache


...

    100/100      4.13G     0.7495     0.4463      0.919        149        640: 100%|██████████| 9/9 [00:02<00:00,  3.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.87it/s]

                   all         20        142      0.923      0.711      0.751      0.491



100 epochs completed in 0.121 hours.
Optimizer stripped from runs/detect/train34/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train34/weights/best.pt, 19.2MB

Validating runs/detect/train34/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.45it/s]


                   all         20        142       0.93      0.787      0.861      0.551
                person          3          6      0.994      0.667       0.85       0.33
                   car         20        128      0.904      0.813      0.926      0.612
                   bus          2          3      0.908      0.667      0.673       0.54
                 truck          2          5      0.915          1      0.995       0.72
Speed: 0.4ms preprocess, 2.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train34


In [ ]:

source = "/content/runs"
destination = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/models/baseline_100/snow"

# If the destination folder already exists and you want to merge/overwrite,
# pass dirs_exist_ok=True (available in Python 3.8+)
shutil.copytree(src=source, dst=destination, dirs_exist_ok=True)

'/content/drive/MyDrive/Info_assurance/models/baseline_100/snow'

## Remove runs

In [ ]:
directory = "/content/runs"

if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Directory '{directory}' has been removed.")
else:
    print(f"Directory '{directory}' does not exist.")

Directory '/content/runs' has been removed.


In [ ]:
# Load the trained/best weights
model_path = "/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/models/baseline_100/snow/detect/train34/weights/best.pt"
model = YOLO(model_path)

In [ ]:
# results = model.predict(source="/content/drive/MyDrive/Info_assurance/split_dawn_data/Fog/test/images",
#                         conf=0.25,
#                         save=True)


In [ ]:
# for r in results:
#     boxes = r.boxes  # list of boxes
#     for box in boxes:
#         cls_id = int(box.cls[0])
#         conf = float(box.conf[0])
#         # x1, y1, x2, y2 = box.xyxy[0]
#         print(f"Detected class {cls_id} with confidence {conf}")


In [ ]:
# Evaluate on test set
metrics = model.val(data="/content/drive/MyDrive/Info_assurance/synthetic_dawn_data/snow.yaml",
                    split="test")  # explicitly evaluate the test split


Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/drive/MyDrive/Info_assurance/split_dawn_data/Snow/test/labels... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:34<00:00,  1.21it/s]

val: New cache created: /content/drive/MyDrive/Info_assurance/split_dawn_data/Snow/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.17it/s]


                   all         42        529      0.712      0.596      0.634       0.42
                person         13         31      0.866      0.624      0.715      0.423
                   car         42        467      0.813      0.774      0.844      0.551
                   bus          4          6      0.653      0.667      0.625      0.472
                 truck         10         25      0.516       0.32      0.351      0.234
Speed: 5.0ms preprocess, 15.5ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val9


In [ ]:
print(metrics)  # or metrics.keys()


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 2, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ce9f99518d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0